In [ ]:
# https://github.com/anjesh/Jupyter-GeoJson-Mapping
# https://www.ryanbaumann.com/blog/2016/4/3/embedding-mapbox-plots-in
#https://github.com/jwass/geojsonio.py-jupyter-notebooks
# http://jupyter-gmaps.readthedocs.io/en/latest/gmaps.html
#https://github.com/planetlabs/planet-client-python

In [ ]:
from planet import api
from planet.api import filters
from sys import stdout
import pandas as pd
import sys
import os
import json
import urllib
import numpy as np
import seaborn as sns
import rasterio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from scipy import ndimage

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from traitlets import link

from IPython.display import display, Image, HTML
%matplotlib inline

# will pick up api_key via environment variable PL_API_KEY
# but can be specified using `api_key` named argument
client = api.ClientV1(api_key=PLANET_API_KEY)
api_keys = json.load(open("apikeys.json",'r'))

In [ ]:
# Planet Basemaps Setup
# Basemap Mosaic (v1 API)
mosaicsSeries = 'global_quarterly_2017q1_mosaic'
# Planet tile server base URL (Planet Explorer Mosaics Tiles)
mosaicsTilesURL_base = 'https://tiles0.planet.com/experimental/mosaics/planet-tiles/' + mosaicsSeries + '/gmap/{z}/{x}/{y}.png'
# Planet tile server url
mosaicsTilesURL = mosaicsTilesURL_base + '?api_key=' + api_keys["PLANET_API_KEY"]
# Map Settings 
# Define colors
colors = {'blue': "#009da5"}
# Define initial map center lat/long
center = [45.5231, -122.6765]
# Define initial map zoom level
zoom = 11
# Set Map Tiles URL
planetMapTiles = TileLayer(url= mosaicsTilesURL)
# Create the map
m = Map(
    center=center, 
    zoom=zoom,
    default_tiles = planetMapTiles # Uncomment to use Planet.com basemap
)
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)
# Initialize an action counter variable
actionCount = 0
AOIs = {}

# Register the draw controls handler
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    actionCount += 1
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    
# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
m

In [ ]:
print AOIs[1]
myAOI = AOIs[1]["geometry"]
# geojson AOI
aoi = {

        "type": "Polygon",
        "coordinates": [
          [
            [
              -123.01528930664061,
              45.213003555993964
            ],
            [
              -122.11441040039061,
              45.213003555993964
            ],
            [
              -122.11441040039061,
              45.82401446834977
            ],
            [
              -123.01528930664061,
              45.82401446834977
            ],
            [
              -123.01528930664061,
              45.213003555993964
            ]
          ]
        ]
}


# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes
query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', gt=0),
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
scenes = []
planet_map = {}
for item in result.items_iter(limit=None):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
scenes = pd.DataFrame(data=scenes)
scenes.head()

In [ ]:
# now let's clean up the datetime stuff
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])

In [ ]:
import datetime 
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"
recent = scenes["acquired"] > datetime.date(year=2017,month=4,day=15)
good_scenes = scenes[(good&clear&recent)]
good_scenes

In [ ]:
imgs = []
for img in good_scenes["thumbnail"].tolist():
    imgs.append(Image(url=img))
display(*imgs)

In [ ]:
to_get = good_scenes["id"].tolist()
client.get_assets(planet_map(to_get[0]))